# Introdução

Este notebook é utilizado para o teste final da rede neural selecionada e treinada. Com isto, será possível classificar o sucesso ou fracasso desta rede em determinar de maneira eficaz a temperatura de melting de proteínas a partir apenas da quantidade de aminoácidos da proteína e os aditivos presentes em solução com a proteína.

## Importação

Antes de começar o código, precisamos importar as funções, bibliotecas e módulos necessários para esta análise

In [1]:
# Função para pegar os aminoácidos de uma proteína e classe base para criação das MLPs
from funcoes import get_amino, MLP

# Módulo para carregar os dados do arquivo de salvamento
import pickle

# Módulo do pytorch para carregar e utilizar a rede
import torch

# Metrica de erro quadrático médio
from sklearn.metrics import mean_squared_error

## Carregando os dados

A primeira etapa é carregar os dados já salvos de teste a serem utilizados, além do normalizador utilizado para tratamento dos dados. Como selecionamos a melhor rede como sendo a que considera dados de input de aditivos do tipo 1 (mais informações: https://github.com/selaxco/Melting-Point-Prediction), podemos pegar apenas os dados referentes a estes inputs

In [2]:
file_data = 'Dados/test_data.pickle'

with open(file_data, 'rb') as file_handler:
    save = pickle.load(file_handler)
    
fn_ativacao = save['fn_ativacao']    

x_teste = save['x_teste1']
y_teste = save['y_teste']

x_norm = save['x_norm1']
y_norm = save['y_norm']

n_entrada = x_teste.shape[1]
c1 = save['c1_mlp']
c2 = save['c2_mlp']
c3 = save['c3_mlp']
n_saida = y_teste.shape[1]

## Carregando a rede

Após carregar os dados, precisamos carregar a rede em si, para isso, podemos utilizar o próprio pytroch, mas antes, devemos definir os dados utilizados anteriormente para a rede, que podemos obter dos dados carregados anteriormente

In [3]:
file_mlp = 'Dados/mlp_save.pt'

mlp = MLP(fn_ativacao, n_entrada, c1, c2, c3, n_saida)
mlp.load_state_dict(torch.load(file_mlp))
mlp.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=21, out_features=30, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=30, out_features=30, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=30, out_features=1, bias=True)
  )
)

## Iniciando os teste finais

Agora podemos começar com o primeiro teste real, testar proteínas separadas especialmente para isto, ou seja, o conjunto de 5% dos dados que foram separados apenas para este teste

In [4]:
with torch.no_grad():
    y_true = y_norm.inverse_transform(y_teste)
    y_pred = mlp(x_teste)
    y_pred = y_norm.inverse_transform(y_pred)

RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 3.992077350616455


Podemos observar que o erro médio do teste real foi maior que o teste do treinamento, porém, esta é uma diferença relativamente pequena, e o erro permaneceu abaixo da faixa de 4°C

## Últimos testes

Os últimos teste a serem conduzidos são utilizando proteínas que não estavam nem mesmo no conjunto de dados utilizado, e para isso, foiram selecionadas duas proteínas, Quinase dependente de Ciclina de Homo Sapiens e a Beta-Catenina de Zebra fish. A primeira se encaixa dentro do limite de 300 aminoácidos e temperatura de melting de 45° em que a rede foi treinada, já a segunda, é um desafio que queremos ver o resultado, se trata de uma proteína com 780 aminoácidos com melting de 52°, quase o triplo do tamanho que a rede está acostumada, mas veremos como ela se sairá. Para o teste, foi feita uma função simples que utiliza os dados de teste e a rede para estimar uma previsão.

In [5]:
def teste_rede(uniprot_id, mlp, additive, normalizador_x, normalizador_y, mlp_type=1):
    '''Retorna a previsão da temperatura de melting da rede com base em uma proteína passada.
    
    Args:
        uniprot_id: string, id da proteína no UniProt;
        
        mlp: torch.Model, rede a ser utilizada para a previsão;
        
        additive: float / list, o input de aditivo a ser utilizado (dependo do tipo da rede);
        
        normalizador_x: normalizador utilizado nos dados de features;
    
        normalizador_y: normalizador utilizado nos dados de targets;
        
        mlp_type (default: 1): int, tipo da rede (aditivos).
    '''
    sequence = get_amino(uniprot_id)
    dados = []
    
    for amino in 'ACDEFGHIKLMNPQRSTVWY':
        dados += [sequence[amino]]
    if mlp_type == 1:
        dados += [additive]
    else:
        dados += [additive[0], additive[1]]
    
    dados_norm = normalizador_x.transform([dados])
    dados_tens = torch.tensor(dados_norm, dtype=torch.float32)
    
    mlp.eval()
    
    with torch.no_grad():
        y_pred = mlp(dados_tens)
        y_pred = normalizador_y.inverse_transform(y_pred)

    print(y_pred[0][0])

## Quinase dependente de Ciclina (Homo Sapiens - 45°C)

In [6]:
teste_rede('P24941', mlp, 0, x_norm, y_norm)

51.493202


Podemos observar que o erro da rede com um dado de uma proteína não antes vista resultou em um erro de aproximadamente 7°C

## Beta-Catenina (Zebra Fish - 52°C)

In [7]:
teste_rede('F1QGH7', mlp, 0, x_norm, y_norm)

58.17991


Podemos observar que o erro da rede com um dado de uma proteína não antes vista, com tamanho maior que as de treino e de outra espécie resultou em um erro de aproximadamente 6°C